# Gallery Example: M/M/1-PS Multiclass Queue

This example demonstrates an M/M/1 queue with Processor Sharing and multiple classes:
- **Arrivals**: Exponential (separate Poisson process per class)
- **Service**: Exponential (class-dependent)
- **Servers**: 1 server
- **Scheduling**: Processor Sharing (PS)
- **Classes**: 2 job classes

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
GlobalConstants.setVerbose(VerboseLevel.STD)

In [ ]:
fun gallery_mm1_ps_multiclass(): Network {
    """Create M[2]/M[2]/1-PS multiclass queueing model"""
    val model = Network("M[2]/M[2]/1-PS")
// Block 1: nodes
    val source = Source(model, "mySource")
    val queue = Queue(model, "myQueue", SchedStrategy.PS)  # Processor Sharing
    val sink = Sink(model, "mySink")
// Block 2: classes
    val oclass1 = OpenClass(model, "myClass1")
    source.setArrival(oclass1, Exp(1))
    queue.setService(oclass1, Exp(4))
    
    val oclass2 = OpenClass(model, "myClass2")
    source.setArrival(oclass2, Exp(0.5))
    queue.setService(oclass2, Exp(4))
// Block 3: topology
    val P = model.initRoutingMatrix()
    P.addRoute(oclass1, source, queue, 1.0)
    P.addRoute(oclass1, queue, sink, 1.0)
    P.addRoute(oclass2, source, queue, 1.0)
    P.addRoute(oclass2, queue, sink, 1.0)
    model.link(P)
    
    return model
// Create the model
val model = gallery_mm1_ps_multiclass()

## About Multiclass Processor Sharing

Under Processor Sharing:
- All jobs in the system share the server capacity equally
- Each of n jobs receives 1/n of capacity, regardless of class
- Response time is proportional to service requirement

**Class 1**: λ₁ = 1, μ₁ = 4 → ρ₁ = 0.25
**Class 2**: λ₂ = 0.5, μ₂ = 4 → ρ₂ = 0.125
**Total utilization**: ρ = 0.375

PS scheduling provides fairness across classes based on service requirements.

In [ ]:
// Solve with multiple solvers
println("\n=== Solver Results ===")
// MVA Solver
val solver_mva = MVA(model)
val avg_table_mva = solver_mva.avgTable
println("\nMVA Solver:")
println(avg_table_mva)
// CTMC Solver
val solver_ctmc = CTMC(model, "cutoff", 15)
val avg_table_ctmc = solver_ctmc.avgTable
println("\nCTMC Solver:")
println(avg_table_ctmc)
// Fluid Solver
val solver_fluid = FLD(model)
val avg_table_fluid = solver_fluid.avgTable
println("\nFluid Solver:")
println(avg_table_fluid)